# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd

if '../../../notebooks/' not in sys.path:
    sys.path.append('../../../notebooks/')

import utils.constants as cts

from models.oface_mouth_model import OpenfaceMouth
from data_loaders.data_loader import DLName
from gt_loaders.gt_names import GTName
from net_trainer import NetworkTrainer, BaseModel

# Start Network Trainer

In [3]:
kwargs = { 
    'use_neptune': False,
    'exp_params' : {
        'name': 'train_mobilenetv2',
        'description': 'Experimenting run with GENKI GT data',
        'tags': ['mobilenetv2', 'ground truth', 'genki'],
        'src_files': ['net_trainer.py']
    },
    'properties': {
#         'tagger_model': OpenfaceMouth(),
#         'dl_names': [DLName.FVC_PYBOSSA],
        'req': cts.ICAO_REQ.MOUTH,
        'aligned': True,
        'use_gt_data': True,
        'gt_names': [GTName.GENKI]
    },
    'net_train_params': {
        'base_model': BaseModel.VGG16,
        'batch_size': 64,
        'n_epochs': 30,
        'early_stopping': 30,
        'shuffle': False,
        'dense_units': 128,
        'learning_rate': 1e-5,
        'optimizer': 'Adam',
        'dropout': 0.25,
        'train_prop': 0.7,
        'validation_prop': 0.15,
        'test_prop': 0.15,
        'seed': 42,
        'validation_split': 0.15
    }
}

trainer = NetworkTrainer(**kwargs)

-----
Use Neptune:  False
-----
Args: 
{'exp_params': {'description': 'Experimenting run with GENKI GT data',
                'name': 'train_mobilenetv2',
                'src_files': ['net_trainer.py'],
                'tags': ['mobilenetv2', 'ground truth', 'genki']},
 'net_train_params': {'base_model': <BaseModel.VGG16: {'target_size': (224, 224), 'prep_function': <function preprocess_input at 0x7f6b3cb3d940>}>,
                      'batch_size': 64,
                      'dense_units': 128,
                      'dropout': 0.25,
                      'early_stopping': 30,
                      'learning_rate': 1e-05,
                      'n_epochs': 30,
                      'optimizer': 'Adam',
                      'seed': 42,
                      'shuffle': False,
                      'test_prop': 0.15,
                      'train_prop': 0.7,
                      'validation_prop': 0.15,
                      'validation_split': 0.15},
 'properties': {'aligned': True,
    

# Load Data

In [6]:
trainer.load_training_data()

Loading data
len(files_list_train):  3990
labels_train.shape: (3990, 1)
Ground Thruth shape:  (3990, 1)
Input data.shape: (3990, 4)
Number of Samples: 3990
Data loaded


# Data Balancing

In [7]:
trainer.balance_input_data()

Balancing input dataset..
df_comp.shape: (1832, 4), df_non_comp.shape: (2158, 4)
final_df.shape:  (3990, 4)
n_comp:  1832
n_non_comp:  2158
Input dataset balanced


# Data Generators

In [8]:
trainer.setup_data_generators()

Starting data generators
Found 2884 validated image filenames.
Found 508 validated image filenames.
Found 598 validated image filenames.
TOTAL: 3990


# Use Neptune

# Training Model

In [11]:
trainer.train_model()

Training VGG16 network
Epoch 1/30
45/45 [==============================] - 7s 152ms/step - loss: 3.3879 - accuracy: 0.4894 - val_loss: 2.4807 - val_accuracy: 0.5089
Epoch 2/30
32/45 [====================>.........] - ETA: 1s - loss: 2.8661 - accuracy: 0.4799

KeyboardInterrupt: 

In [12]:
trainer.model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

# Plots

In [ ]:
trainer.draw_training_history()

# Saving Trained Model

# Test Trained Model

In [ ]:
trainer.test_model()

# Evaluate Model

In [ ]:
trainer.evaluate_model()

# Vizualize Model Classification

In [ ]:
trainer.vizualize_predictions()

# Finishing Experiment Manager

In [ ]:
trainer.finish_experiment()